# The Battle of the Neighbourhoods: Where should I stay?

### Let's import all the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import math

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
# from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
# from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
# import folium
print('Libraries imported.')

Libraries imported.


### First we identify the coordinates of the Work Location

In [2]:
work_address = '55 Pasir Panjang Road, Singapore'

geolocator = Nominatim(user_agent="SG_explorer")
work_location = geolocator.geocode(work_address)
work_latitude = work_location.latitude
work_longitude = work_location.longitude
print('The geograpical coordinate of my work location are {}, {}.'.format(work_latitude, work_longitude))

The geograpical coordinate of my work location are 1.277634, 103.7883346.


### Now we load the Singapore Areas List to Pandas DataFrame

In [3]:
url='https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore' #Wikipedia Page URL
SGPA=pd.read_html(url) # Reads the webpage to a list

In [4]:
SG_Areas=SGPA[2] # Dataframe we need is the third element of the list
SG_Areas.head() # Checking if importing worked fine

,Name (English),Malay,Chinese,Pinyin,Tamil,Region,Area (km2),Population[7],Density (/km2)
0,Ang Mo Kio,NaN,宏茂桥,Hóng mào qiáo,ஆங் மோ கியோ,North-East,13.94,163950,13400
1,Bedok,*,勿洛,Wù luò,பிடோக்,East,21.69,279380,13000
2,Bishan,NaN,碧山,Bì shān,பீஷான்,Central,7.62,88010,12000
3,Boon Lay,NaN,文礼,Wén lǐ,பூன் லே,West,8.23,30,3.6
4,Bukit Batok,*,武吉巴督,Wǔjí bā dū,புக்கிட் பாத்தோக்,West,11.13,153740,14000


In [5]:
SG_Areas.drop(['Malay', 'Chinese', 'Pinyin', 'Tamil', 'Area (km2)', 'Population[7]', 'Density (/km2)'], axis=1, inplace = True)
SG_Areas.head()

,Name (English),Region
0,Ang Mo Kio,North-East
1,Bedok,East
2,Bishan,Central
3,Boon Lay,West
4,Bukit Batok,West


In [6]:
SG_Areas.columns=['Name', 'Region']
SG_Areas.head()

,Name,Region
0,Ang Mo Kio,North-East
1,Bedok,East
2,Bishan,Central
3,Boon Lay,West
4,Bukit Batok,West


### Initializing Foursquare Credentials

In [10]:
CLIENT_ID = '0ZPU1H3ST2TYYZHI0TLPSZ3SHHVSBFELN0ECW3T4SRL1IO0T' # your Foursquare ID
CLIENT_SECRET = '4FBXS0Z1O1ODMWP2LGNI2BZ1D3XMRM4GYPTCFX1KFJXDYG1A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0ZPU1H3ST2TYYZHI0TLPSZ3SHHVSBFELN0ECW3T4SRL1IO0T
CLIENT_SECRET:4FBXS0Z1O1ODMWP2LGNI2BZ1D3XMRM4GYPTCFX1KFJXDYG1A


### Loading Coordinates of all Singapore Areas

In [9]:
SG_Areas[['Latitude']]=pd.DataFrame(np.zeros(len(SG_Areas.index)), columns=['Latitude'])
SG_Areas[['Longitude']]=pd.DataFrame(np.zeros(len(SG_Areas.index)), columns=['Longitude'])
SG_Areas.head()

,Name,Region,Latitude,Longitude
0,Ang Mo Kio,North-East,0.0,0.0
1,Bedok,East,0.0,0.0
2,Bishan,Central,0.0,0.0
3,Boon Lay,West,0.0,0.0
4,Bukit Batok,West,0.0,0.0


In [11]:
for i in range(len(SG_Areas.index)):
    address = SG_Areas.iloc[i]['Name']+', Singapore'
    geolocator = Nominatim(user_agent="SG_explorer")
    Area_location = geolocator.geocode(address)
    SG_Areas.iat[i, 2] = Area_location.latitude
    SG_Areas.iat[i, 3] = Area_location.longitude

In [12]:
SG_Areas.head()

,Name,Region,Latitude,Longitude
0,Ang Mo Kio,North-East,1.370080,103.849523
1,Bedok,East,1.323976,103.930216
2,Bishan,Central,1.350986,103.848255
3,Boon Lay,West,1.338550,103.705812
4,Bukit Batok,West,1.349057,103.749591


In [15]:
SG_Areas[['Work Distance']]=pd.DataFrame(np.zeros(len(SG_Areas.index)), columns=['Work Distance'])
SG_Areas.head()

,Name,Region,Latitude,Longitude,Work Distance
0,Ang Mo Kio,North-East,1.370080,103.849523,0.0
1,Bedok,East,1.323976,103.930216,0.0
2,Bishan,Central,1.350986,103.848255,0.0
3,Boon Lay,West,1.338550,103.705812,0.0
4,Bukit Batok,West,1.349057,103.749591,0.0


### Here we find distance to work based on coordiates. Since only comparison is required, exact kms are not required.

In [18]:
for i in range(len(SG_Areas.index)):
    SG_Areas.iat[i, 4] = math.sqrt((SG_Areas.iloc[i]['Latitude']-work_latitude)**2+(SG_Areas.iloc[i]['Longitude']-work_longitude)**2)
SG_Areas.head()

,Name,Region,Latitude,Longitude,Work Distance
0,Ang Mo Kio,North-East,1.370080,103.849523,0.110862
1,Bedok,East,1.323976,103.930216,0.149258
2,Bishan,Central,1.350986,103.848255,0.094715
3,Boon Lay,West,1.338550,103.705812,0.102571
4,Bukit Batok,West,1.349057,103.749591,0.081255


In [19]:
max_work_dist = SG_Areas['Work Distance'].max()
min_work_dist = SG_Areas['Work Distance'].min()
print('The closest area is {} degs away and farthest is {} degs away'.format(min_work_dist, max_work_dist))

The closest area is 0.024535144873425965 degs away and farthest is 0.28271221800140844 degs away


### Extracting all nearby venues for all areas in Singapore

In [13]:
def getNearbyVenues(area_name, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(area_name, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'], 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Name', 
                  'Latitude', 
                  'Longitude', 
                  'Venue ID', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

SG_venues = getNearbyVenues(area_name=SG_Areas['Name'],
                                   latitudes=SG_Areas['Latitude'],
                                   longitudes=SG_Areas['Longitude']
                                  )

Ang Mo Kio
Bedok
Bishan
Boon Lay
Bukit Batok
Bukit Merah
Bukit Panjang
Bukit Timah
Central Water Catchment
Changi
Changi Bay
Choa Chu Kang
Clementi
Downtown Core
Geylang
Hougang
Jurong East
Jurong West
Kallang
Lim Chu Kang
Mandai
Marina East
Marina South
Marine Parade
Museum
Newton
North-Eastern Islands
Novena
Orchard
Outram
Pasir Ris
Paya Lebar
Pioneer
Punggol
Queenstown
River Valley
Rochor
Seletar
Sembawang
Sengkang
Serangoon
Simpang
Singapore River
Southern Islands
Straits View
Sungei Kadut
Tampines
Tanglin
Tengah
Toa Payoh
Tuas
Western Islands
Western Water Catchment
Woodlands
Yishun


In [20]:
SG_venues.head()

,Name,Latitude,Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ang Mo Kio,1.37008,103.849523,4bc30123abf495215220c393,Old Chang Kee,1.369094,103.848389,Snack Place
1,Ang Mo Kio,1.37008,103.849523,4b15f661f964a52012b623e3,FairPrice Xtra,1.369279,103.848886,Supermarket
2,Ang Mo Kio,1.37008,103.849523,4b1ea04ff964a520e21c24e3,MOS Burger,1.369170,103.847831,Burger Joint
3,Ang Mo Kio,1.37008,103.849523,4b5eef6df964a5208b9e29e3,PLAYe,1.369109,103.848225,Hobby Shop
4,Ang Mo Kio,1.37008,103.849523,50712b67e4b0548504446dea,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House


In [21]:
SG_Venues_cat=pd.DataFrame(SG_venues['Venue Category'].unique(), columns=['Category'])
SG_Venues_cat

,Category
0,Snack Place
1,Supermarket
2,Burger Joint
3,Hobby Shop
4,Noodle House
5,Fast Food Restaurant
6,Sandwich Place
7,Dessert Shop
8,Asian Restaurant
9,Japanese Restaurant


In [22]:
SG_Venues_cat[['User Weights']]=pd.DataFrame(np.zeros((227)), columns=['User Weights'])
SG_Venues_cat.head()

,Category,User Weights
0,Snack Place,0.0
1,Supermarket,0.0
2,Burger Joint,0.0
3,Hobby Shop,0.0
4,Noodle House,0.0


### Now we export and get feedback from user on how much they value each category

In [98]:
file_loc = r'http://ambujsaxena.com/User_Weights.xlsx'
user_feedback=pd.read_excel(file_loc)

In [99]:
user_feedback.set_index('Category', inplace=True)
user_feedback.head()

,User Weights
Category,
Snack Place,0
Supermarket,5
Burger Joint,0
Noodle House,0
Hobby Shop,0


In [100]:
weight_total=user_feedback['User Weights'].sum()
weight_total

55

In [26]:
SG_venues[['Ratings']]=pd.DataFrame(np.zeros(len(SG_venues.index)), columns=['User Weights'])
SG_venues.head()

,Name,Latitude,Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category,Ratings
0,Ang Mo Kio,1.37008,103.849523,4bc30123abf495215220c393,Old Chang Kee,1.369094,103.848389,Snack Place,0.0
1,Ang Mo Kio,1.37008,103.849523,4b15f661f964a52012b623e3,FairPrice Xtra,1.369279,103.848886,Supermarket,0.0
2,Ang Mo Kio,1.37008,103.849523,4b1ea04ff964a520e21c24e3,MOS Burger,1.369170,103.847831,Burger Joint,0.0
3,Ang Mo Kio,1.37008,103.849523,4b5eef6df964a5208b9e29e3,PLAYe,1.369109,103.848225,Hobby Shop,0.0
4,Ang Mo Kio,1.37008,103.849523,50712b67e4b0548504446dea,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House,0.0


### Since we have Personal Account Tier, we need to ensure number of premium calls is less than 500

In [27]:
queries_to_make=0

for cat in SG_venues['Venue Category']:
    if (user_feedback.loc[cat]['User Weights']>0):
        queries_to_make=queries_to_make+1
        
queries_to_make

407

In [42]:
SG_venues

,Name,Latitude,Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category,Ratings
0,Ang Mo Kio,1.370080,103.849523,4bc30123abf495215220c393,Old Chang Kee,1.369094,103.848389,Snack Place,7.7
1,Ang Mo Kio,1.370080,103.849523,4b15f661f964a52012b623e3,FairPrice Xtra,1.369279,103.848886,Supermarket,7.6
2,Ang Mo Kio,1.370080,103.849523,4b1ea04ff964a520e21c24e3,MOS Burger,1.369170,103.847831,Burger Joint,7.3
3,Ang Mo Kio,1.370080,103.849523,4b5eef6df964a5208b9e29e3,PLAYe,1.369109,103.848225,Hobby Shop,7.2
4,Ang Mo Kio,1.370080,103.849523,50712b67e4b0548504446dea,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House,7.5
5,Ang Mo Kio,1.370080,103.849523,5d48d8dfdae4eb0008d85e5a,A&W,1.369541,103.849043,Fast Food Restaurant,7.0
6,Ang Mo Kio,1.370080,103.849523,4ece4dd693ad987b4a9fb4ef,NTUC FairPrice,1.371507,103.847082,Supermarket,7.3
7,Ang Mo Kio,1.370080,103.849523,4bceddaeb6c49c7410bb9791,Subway,1.369136,103.847612,Sandwich Place,6.9
8,Ang Mo Kio,1.370080,103.849523,5213643e11d236c03cd2810f,Swensen's,1.369112,103.847718,Dessert Shop,6.9
9,Ang Mo Kio,1.370080,103.849523,53c68f51498efd710da56841,e品红麻辣香锅 Mala Hot Pot,1.371498,103.847803,Asian Restaurant,6.9


### Extracting Ratings for Foursquare. Error handling routine required since not all venues have ratings

In [59]:
for cat in range(len(SG_venues.index)):
    if (user_feedback.loc[SG_venues.iloc[cat][7]]['User Weights']>0):
        venue_id = SG_venues.iloc[cat][3]
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        result = requests.get(url).json()
        try:
            SG_venues.iat[cat, 8]=result['response']['venue']['rating']
        except:
            print(cat, ' resulted in error!')

98  resulted in error!
100  resulted in error!
101  resulted in error!
147  resulted in error!
206  resulted in error!
207  resulted in error!
208  resulted in error!
210  resulted in error!
222  resulted in error!
225  resulted in error!
252  resulted in error!
283  resulted in error!
307  resulted in error!
388  resulted in error!
389  resulted in error!
567  resulted in error!
635  resulted in error!
693  resulted in error!
694  resulted in error!
699  resulted in error!
700  resulted in error!
703  resulted in error!
893  resulted in error!
895  resulted in error!
1027  resulted in error!
1029  resulted in error!
1104  resulted in error!
1106  resulted in error!
1108  resulted in error!
1119  resulted in error!
1183  resulted in error!
1266  resulted in error!
1282  resulted in error!
1316  resulted in error!
1550  resulted in error!
1731  resulted in error!


In [61]:
SG_venues

,Name,Latitude,Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category,Ratings
0,Ang Mo Kio,1.370080,103.849523,4bc30123abf495215220c393,Old Chang Kee,1.369094,103.848389,Snack Place,7.7
1,Ang Mo Kio,1.370080,103.849523,4b15f661f964a52012b623e3,FairPrice Xtra,1.369279,103.848886,Supermarket,7.6
2,Ang Mo Kio,1.370080,103.849523,4b1ea04ff964a520e21c24e3,MOS Burger,1.369170,103.847831,Burger Joint,7.3
3,Ang Mo Kio,1.370080,103.849523,4b5eef6df964a5208b9e29e3,PLAYe,1.369109,103.848225,Hobby Shop,7.2
4,Ang Mo Kio,1.370080,103.849523,50712b67e4b0548504446dea,Face Ban Mian 非板面 (Ang Mo Kio),1.372031,103.847504,Noodle House,7.5
5,Ang Mo Kio,1.370080,103.849523,5d48d8dfdae4eb0008d85e5a,A&W,1.369541,103.849043,Fast Food Restaurant,7.0
6,Ang Mo Kio,1.370080,103.849523,4ece4dd693ad987b4a9fb4ef,NTUC FairPrice,1.371507,103.847082,Supermarket,7.3
7,Ang Mo Kio,1.370080,103.849523,4bceddaeb6c49c7410bb9791,Subway,1.369136,103.847612,Sandwich Place,6.9
8,Ang Mo Kio,1.370080,103.849523,5213643e11d236c03cd2810f,Swensen's,1.369112,103.847718,Dessert Shop,6.9
9,Ang Mo Kio,1.370080,103.849523,53c68f51498efd710da56841,e品红麻辣香锅 Mala Hot Pot,1.371498,103.847803,Asian Restaurant,6.9


### Finding which categories have non-zero rating to add to our work sheet

In [67]:
user_feedback.sort_values(by='User Weights', ascending=False, inplace = True)
user_feedback

,User Weights
Category,
Restaurant,10
Supermarket,5
Shopping Mall,5
Bank,5
Department Store,4
Multiplex,4
Convenience Store,4
Bakery,4
Pharmacy,3


In [68]:
user_feedback = user_feedback.head(13)
user_feedback

,User Weights
Category,
Restaurant,10
Supermarket,5
Shopping Mall,5
Bank,5
Department Store,4
Multiplex,4
Convenience Store,4
Bakery,4
Pharmacy,3


In [86]:
user_feedback.index

Index(['Restaurant', 'Supermarket', 'Shopping Mall', 'Bank',
       'Department Store', 'Multiplex', 'Convenience Store', 'Bakery',
       'Pharmacy', 'Fast Food Restaurant', 'Chinese Restaurant', 'Food Court',
       'Pizza Place'],
      dtype='object', name='Category')

In [90]:
SG_Areas_Sheet = pd.concat([SG_Areas, pd.DataFrame(np.zeros((len(SG_Areas.index), len(user_feedback.index))), columns = user_feedback.index)], axis=1, sort=False)
SG_Areas_Sheet.head()

,Name,Region,Latitude,Longitude,Work Distance,Restaurant,Supermarket,Shopping Mall,Bank,Department Store,Multiplex,Convenience Store,Bakery,Pharmacy,Fast Food Restaurant,Chinese Restaurant,Food Court,Pizza Place
0,Ang Mo Kio,North-East,1.370080,103.849523,0.110862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bedok,East,1.323976,103.930216,0.149258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bishan,Central,1.350986,103.848255,0.094715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Boon Lay,West,1.338550,103.705812,0.102571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bukit Batok,West,1.349057,103.749591,0.081255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Now we find out how many venues from each category are in vicinity of different areas of Singapore

In [116]:
for i in range(len(SG_venues.index)):
    try:
        if ( (user_feedback.loc[SG_venues.loc[i]['Venue Category']]['User Weights']>0) and (SG_venues.loc[i]['Ratings']>7.0) ):
            try:
                if (SG_Areas_Sheet.loc[SG_venues.loc[i]['Name']][SG_venues.loc[i]['Venue Category']]<5.0):
                    SG_Areas_Sheet.at[SG_venues.loc[i]['Name'], SG_venues.loc[i]['Venue Category']] = SG_Areas_Sheet.at[SG_venues.loc[i]['Name'], SG_venues.loc[i]['Venue Category']] + 1
            except:
                print('Error in i={}, j={}'.format(i, j))
    except:
        print('Error in i={}'.format(i))

In [117]:
SG_Areas_Sheet

,Region,Latitude,Longitude,Work Distance,Restaurant,Supermarket,Shopping Mall,Bank,Department Store,Multiplex,Convenience Store,Bakery,Pharmacy,Fast Food Restaurant,Chinese Restaurant,Food Court,Pizza Place
Name,,,,,,,,,,,,,,,,,
Ang Mo Kio,North-East,1.370080,103.849523,0.110862,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bedok,East,1.323976,103.930216,0.149258,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0
Bishan,Central,1.350986,103.848255,0.094715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
Boon Lay,West,1.338550,103.705812,0.102571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
Bukit Batok,West,1.349057,103.749591,0.081255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bukit Merah,Central,1.270439,103.828318,0.040626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bukit Panjang,West,1.379149,103.761413,0.105024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bukit Timah,Central,1.354690,103.776372,0.077979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Central Water Catchment,North,1.375708,103.801743,0.098987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Now we add final score column to rank each area

In [121]:
SG_Areas_Sheet[['Score']]=pd.DataFrame(np.zeros((len(SG_Areas_Sheet.index))), columns=['Score'])
SG_Areas_Sheet.head()

,Region,Latitude,Longitude,Work Distance,Restaurant,Supermarket,Shopping Mall,Bank,Department Store,Multiplex,Convenience Store,Bakery,Pharmacy,Fast Food Restaurant,Chinese Restaurant,Food Court,Pizza Place,Score
Name,,,,,,,,,,,,,,,,,,
Ang Mo Kio,North-East,1.370080,103.849523,0.110862,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Bedok,East,1.323976,103.930216,0.149258,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,NaN
Bishan,Central,1.350986,103.848255,0.094715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,NaN
Boon Lay,West,1.338550,103.705812,0.102571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
Bukit Batok,West,1.349057,103.749591,0.081255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [126]:
for i in range(len(SG_Areas_Sheet.index)):
    region_score = 0
    region_score = region_score + 10*(max_work_dist - SG_Areas_Sheet.iloc[i][3])/(max_work_dist - min_work_dist)   # min_work_dist, max_work_dist
    for j in range(13):
        region_score = region_score + SG_Areas_Sheet.iloc[i][j + 4]*user_feedback.loc[SG_Areas_Sheet.columns[4]]['User Weights']
    SG_Areas_Sheet.iat[i, 17] = region_score
SG_Areas_Sheet.head()

,Region,Latitude,Longitude,Work Distance,Restaurant,Supermarket,Shopping Mall,Bank,Department Store,Multiplex,Convenience Store,Bakery,Pharmacy,Fast Food Restaurant,Chinese Restaurant,Food Court,Pizza Place,Score
Name,,,,,,,,,,,,,,,,,,
Ang Mo Kio,North-East,1.370080,103.849523,0.110862,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.656305
Bedok,East,1.323976,103.930216,0.149258,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,85.169095
Bishan,Central,1.350986,103.848255,0.094715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,27.281709
Boon Lay,West,1.338550,103.705812,0.102571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,36.977446
Bukit Batok,West,1.349057,103.749591,0.081255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.803065


### Now we sort the list in descending order of score to find the best areas to live in

In [127]:
SG_Areas_Sheet.sort_values(by='Score', ascending=False, inplace = True)
SG_Areas_Sheet.head()

,Region,Latitude,Longitude,Work Distance,Restaurant,Supermarket,Shopping Mall,Bank,Department Store,Multiplex,Convenience Store,Bakery,Pharmacy,Fast Food Restaurant,Chinese Restaurant,Food Court,Pizza Place,Score
Name,,,,,,,,,,,,,,,,,,
Orchard,Central,1.303427,103.831342,0.050149,1.0,1.0,5.0,0.0,3.0,1.0,0.0,5.0,0.0,1.0,4.0,1.0,1.0,239.007908
Jurong East,West,1.333108,103.742294,0.072091,0.0,1.0,2.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,128.158014
Marine Parade,Central,1.302689,103.907395,0.121668,0.0,1.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,96.237731
Bedok,East,1.323976,103.930216,0.149258,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,2.0,0.0,85.169095
Paya Lebar,East,1.317480,103.892353,0.111388,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,2.0,0.0,76.635901


In [133]:
print('The ranking of best places for you to stay based on your requirements are the following (with their scores in brackets):')
for i in range(5):
    print(i+1, SG_Areas_Sheet.index[i], '(%3.2f)'%(SG_Areas_Sheet.iloc[i][17]))

The ranking of best places for you to stay based on your requirements are the following (with their scores in brackets):
1 Orchard (239.01)
2 Jurong East (128.16)
3 Marine Parade (96.24)
4 Bedok (85.17)
5 Paya Lebar (76.64)


### That's all folks!